# 1. Preparation before running plans... 

## 1.1. Import Numpy and Matplotlib

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

## 1.2. Create a RunEngine:

In [60]:
from bluesky import RunEngine
RE = RunEngine({})

## 1.3. Prepare Live Visualization¶

In [61]:
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)

# Make plots update live while scans run.
from bluesky.utils import install_kicker
install_kicker()

## 1.4. Prepare Data Storage

In [62]:
from databroker import Broker
db = Broker.named('temp')

# Insert all metadata/data captured into db.
RE.subscribe(db.insert)

1

##### This example makes a temporary database. Do not use it for important data. The data will become difficult to access once Python exits or the variable db is deleted. Running Broker.named('temp') a second time creates a fresh, separate temporary database.

## 1.5. Add a Progress Bar

In [63]:
from bluesky.utils import ProgressBarManager
RE.waiting_hook = ProgressBarManager()

# 2. Flow Experiment Plan

In [6]:
from ophyd import Component as Cpt, Device, EpicsMotor, EpicsSignal, EpicsSignalRO

Use simpleIOC in caproto as simulated EPICS

$ python -m caproto.ioc_examples.simple --prefix="simple:ultra:" --list-pvs

$ python -m caproto.ioc_examples.simple --prefix="simple:dds:" --list-pvs

$ python -m caproto.ioc_examples.simple --prefix="simple:shutter:" --list-pvs

In [140]:
## Create a class of EPICS device for syringe pumps, UVVSI(shutter, detector)

class Syringepump(Device):
    rate = Cpt(EpicsSignal, 'A')
    pump = Cpt(EpicsSignal, 'B')
    pump_on = 1
    pump_off = 0

ultra1_p1 = Syringepump('simple:ultra:', name='ultra1_p1')
# ultra1_p2 = Syringepump('simple:ultra:', name='ultra1_p2')

dds1_p1 = Syringepump('simple:dds:', name='dds1_p1')
dds1_p2 = Syringepump('simple:dds:', name='dds1_p2')


# ultra2_p1 = Syringepump('simple:ultra:', name='ultra2_p1')
# ultra2_p2 = Syringepump('simple:ultra:', name='ultra2_p2')

# dds2_p1 = Syringepump('simple:dds:', name='dds2_p1')
# dds2_p2 = Syringepump('simple:dds:', name='dds2_p2')


class UV_shutter(Device):
    open_cmd = Cpt(EpicsSignal, 'A')
    open_val = 1

    close_cmd = Cpt(EpicsSignal, 'B')
    close_val = 0

UV_shutter_e = UV_shutter('simple:shutter:', name='UV_shutter_e')  ## For emission
UV_shutter_a = UV_shutter('simple:shutter:', name='UV_shutter_a')  ## For absorption


### if detector is a ready-to-use EPICS device
# det_e = EpicsMotor('detector_emission_PV', name='det_e')
# det_a = EpicsMotor('detector_absorption_PV', name='det_a')

### For test, use ophyd.sim to simulate two detectors
from ophyd.sim import det1 as det_e, det2 as det_a


In [125]:
live_talbe_list = ['ultra1_p1_rate', 'ultra1_p1_pump',
                   'ultra1_p2_rate', 'ultra1_p2_pump',
                   'dds1_p1_rate', 'dds1_p1_pump',
                   'dds1_p2_rate', 'dds1_p2_pump',
                  ]

In [126]:
## Check if the EPICS devices are connected.

ultra1_p1.wait_for_connection()
# ultra1_p2.wait_for_connection()
dds1_p1.wait_for_connection()
dds1_p2.wait_for_connection()

UV_shutter_a.wait_for_connection()
UV_shutter_e.wait_for_connection()

In [129]:
from bluesky import RunEngine
RE = RunEngine({})

In [130]:
from bluesky.callbacks import LiveTable
# token = RE.subscribe(LiveTable(live_talbe_list))

# # Insert all metadata/data captured into db.
RE.subscribe(db.insert)
RE.waiting_hook = ProgressBarManager()

In [185]:
## Test the simulated EPICS by taking a count

from bluesky.plans import count
device = [dds1_p2.rate, dds1_p1.rate]
RE(count(device, num=3, delay=1), LiveTable(device))



+-----------+------------+--------------+--------------+
|   seq_num |       time | dds1_p2_rate | dds1_p1_rate |
+-----------+------------+--------------+--------------+
|         1 | 19:24:29.5 |            7 |            7 |
|         2 | 19:24:30.5 |            7 |            7 |
|         3 | 19:24:31.5 |            7 |            7 |
+-----------+------------+--------------+--------------+
generator count ['452be370'] (scan num: 2)




('452be370-5d36-487c-8eea-6e9f5159e274',)

In [132]:
## Check if the data are save in databroker

header = db[-1]
header.table()

,time,dds1_p2_rate,dds1_p1_rate
seq_num,,,
1,2022-02-17 22:33:52.789866924,1,1
2,2022-02-17 22:33:53.783720016,1,1
3,2022-02-17 22:33:54.785616875,1,1


In [170]:
## Set flow rate

# Method 1:
ultra1_p1.rate.set(5).wait(timeout=1)

# Method 2:
import bluesky.plan_stubs as bps
RE(bps.abs_set(ultra1_p1.rate, 15, wait=True))

()

In [167]:
ultra1_p1.name

'ultra1_p1'

In [139]:
## Start or stop the syringe pump

# Method 1: if the device can be triggered
ultra1_p1.trigger()
RE(bps.trigger(ultra1_p1, wait=True))

# Method 2: if there is a PV to strart/stop the pump
# such as 'simple:ultra:B' Assume set the value to 1 is on; to 0 is off 
ultra1_p1.pump.set(ultra1_p1.pump_off).wait(timeout=1)
RE(bps.abs_set(ultra1_p1.pump, ultra1_p1.pump_off, wait=True))

()

In [171]:
ultra1_p1.rate.get()
getattr(ultra1_p1, 'rate').get()

15

In [179]:
### Define a plan to set all flow rates



rates = {'ultra1_p1': 2,
         #'ultra1_p2': 3,
         # 'ultra2_p1': 4,
         #'ultra2_p2': 5,
         'dds1_p1': 10,
         'dds1_p2': 7,
         # 'dds2_p1': 8,
         # 'dds2_p2': 9,
        }

def set_pump_rate(rates):
    for i in rates.keys():
        eval(i).rate.set(rates[i]).wait(timeout=1)

# ultra1_p1.rate.set(rates['ultra1_p1']).wait(timeout=1)
# ultra1_p1.rate.set(rates['ultra1_p1']).wait(timeout=1)
# dds1_p1.rate.set(rates['dds1_p1']).wait(timeout=1)
# dds1_p2.rate.set(rates['dds1_p2']).wait(timeout=1)

In [180]:
set_pump_rate(rates)

In [186]:
for i in rates.keys():
    print(eval(i).rate.get())

2
7
7


In [182]:
rates['dds1_p1']

10

In [184]:
dds1_p1.rate.get()

7